In [1]:
from DATA225utils import make_connection, dataframe_query

In [2]:
conn = make_connection(config_file = 'school.ini')
cursor = conn.cursor()

Exception: Configuration file 'school.ini' doesn't exist.

## Nested `SELECT`

#### Who are the classmates of Tim Novak?

In [ ]:
_, df = dataframe_query(conn, 
    """
    SELECT s.first, s.last, c.code, c.subject
    FROM student s, takes t, class c
    WHERE c.code IN (
        SELECT c.code 
        FROM student s, class c, takes t
        WHERE s.first = "Tim"
        AND   s.last = "Novak"
        AND   s.id = t.student_id
        AND   c.code = t.class_code
    )
    AND (c.code = t.class_code)
    AND (s.id = student_id)
    AND NOT (    (s.first = "Tim")
             AND (s.last = "Novak"))
    """
                       )

df

## Use of a view instead

#### Create a view from the nested `SELECT`. 

In [ ]:
cursor.execute('DROP VIEW IF EXISTS class_codes_of_novak')

cursor.execute( 
    """
    CREATE VIEW class_codes_of_novak AS
    SELECT c.code 
    FROM student s, class c, takes t
    WHERE s.first = "Tim"
    AND   s.last = "Novak"
    AND   s.id = t.student_id
    AND   c.code = t.class_code
    """
              )

In [ ]:
_, df = dataframe_query(conn, 'SELECT * FROM class_codes_of_novak')

df

In [ ]:
_, df = dataframe_query(conn, 
    """
    SELECT s.first, s.last, c.code, c.subject
    FROM student s, takes t, class c
    WHERE c.code IN (
        SELECT * FROM class_codes_of_novak
    )
    AND (c.code = t.class_code)
    AND (s.id = student_id)
    AND NOT (    (s.first = "Tim")
             AND (s.last = "Novak"))
    """
                       )

df

## A better use of the view

In [ ]:
_, df = dataframe_query(conn, 
    """
    SELECT s.first, s.last, c.code, c.subject
    FROM student s, takes t, class c, 
         class_codes_of_novak ccn
    WHERE (c.code = ccn.code)
    AND (c.code = t.class_code)
    AND (s.id = student_id)
    AND NOT (    (s.first = "Tim")
             AND (s.last = "Novak"))
    """
                       )

df

## A classmates function

#### In this case, we don't bother with creating a view since we are inserting the student's first and last names into the query.

In [ ]:
def classmates_of(first, last):
    _, df = dataframe_query(conn, 
        """
        SELECT s.first, s.last, c.code, c.subject
        FROM student s, takes t, class c
        WHERE c.code IN (
            SELECT c.code 
            FROM student s, class c, takes t
            """
          f'WHERE s.first = "{first}"'
          f'AND   s.last = "{last}"'
            """
            AND   s.id = t.student_id
            AND   c.code = t.class_code
        )
        AND (c.code = t.class_code)
        AND (s.id = student_id)
        """
      f'AND NOT (    (s.first = "{first}")'
      f'         AND (s.last = "{last}"))'
                       )
                       
    return df

In [ ]:
classmates_of('Tim', 'Novak')

In [ ]:
cursor.execute('SELECT first, last FROM student')

students = cursor.fetchall()
students

In [ ]:
for first, last in students:
    print()
    print(f"Classmates of {first + ' ' + last}:")
    
    display(classmates_of(first, last))

In [ ]:
cursor.close()
conn.close()